# i-TED E characterization - CMAM
## Based on v5 from Multi iTED/v1 from iTED-E

In [1]:
pkg_ver = lambda pkg: "{:<20}{:}".format(pkg.__name__,pkg.__version__)

# ROOT
import uproot
print(pkg_ver(uproot))
import ROOT

# Machine Learning
import sklearn
print(pkg_ver(sklearn))
import torch
print(pkg_ver(torch))

# Data science
import scipy
print(pkg_ver(scipy))
import numpy
print(pkg_ver(numpy))
import pandas
print(pkg_ver(pandas))

# Visualizations
import matplotlib
print(pkg_ver(matplotlib))
import matplotlib.pyplot as plt

import tqdm
print(pkg_ver(tqdm))

import glob

uproot              4.3.5
Welcome to JupyROOT 6.28/04
sklearn             1.2.2
torch               2.0.1
scipy               1.10.1
numpy               1.24.3
pandas              1.5.3
matplotlib          3.7.1
tqdm                4.65.0


In [2]:
%jsroot

In [3]:
class spectrum:
    
    def __init__(self, File_, iTED_, Crystal_, Configuration_, Calibration_):  

        self.__File = ROOT.TFile.Open(File_,"READ")
        self.__iTED = iTED_
        self.__Crystal = Crystal_
        self.__Configuration = Configuration_
        self.__Calibration = Calibration_
        
    def __call__(self, ch):
        return numpy.polyval(self.__Calibration[::-1],ch)
    
    def __ch__(self, en):
        p = numpy.poly1d(self.__Calibration[::-1])
        temp  = set(i for i in (p - en).roots if i > 0)
        return list(temp).pop()

    def ch(self, en):
        p = numpy.poly1d(self.__Calibration[::-1])
        temp  = set(i for i in (p - en).roots if i > 0)
        return list(temp).pop()

    def File(self):
        return self.__File

    def TH1D(self):

        i = self.__iTED
        c = self.__Crystal
        
        self.__TH1D = self.__File.Get(
                            "{}_{}_amplitude_spectra;1".format(
                                "SCATTERER" if c==0 else "ABSORBER",
                                i if c==0 else "{}_{}".format(i,c)
                            )
                        )
        
        return self.__TH1D
    
    def Calibration(self):
        return self.__Calibration
    
    def iTED(self):
        return self.__iTED
    
    def Crystal(self):
        return self.__Crystal
    
    def Configuration(self):
        return self.__Configuration
    
    def Rate(self, Time_):
        return self.__TH1D.Integral()/Time_
    
    def Alpha(self, Time_):
        return self.__TH1D.Integral(
            self.__TH1D.FindBin(self.__ch__(1600)),
            self.__TH1D.FindBin(self.__ch__(2800))
        )/Time_
    
    def __repr__(self):
        return "iTED: {},Crystal: {},Configuration: {}".format(
            self.__iTED,
            self.__Crystal,
            self.__Configuration,
        )
    
    def __str__(self):
        return "{}.{}.{}".format(
            self.__iTED,
            self.__Crystal,
            self.__Configuration,
        )

In [4]:
calp = glob.glob('/run/media/bgameiro/d043b5e4-57a4-457e-8839-cb1adc9c72bc/Data/CMAM2023/Calibration/**/*.CALp', recursive=True)
    
calibrations_df = pandas.read_csv("iTED_E_calibration_CMAM.csv")

In [5]:
file = "/run/media/bgameiro/d043b5e4-57a4-457e-8839-cb1adc9c72bc/Data/CMAM2023/data_2023_06_15/Na22_pet_D.2023_06_15_T.12_54_31_C.itedE_CMAM_4.0v_887C_180s.singles.auto.root"

In [6]:
def get_resolution(cell):
    
    TH1D = cell.TH1D()

    TH1D.GetXaxis().SetRange(TH1D.FindBin(cell.ch(cell.Configuration()*.8)),TH1D.FindBin(cell.ch(cell.Configuration()*1.35)))
    MaxBin   = TH1D.FindBin(TH1D.GetMaximumBin())
    
    ADC_Low  = MaxBin-50
    ADC_High = MaxBin+40

    p1 = 60 if (cell.iTED()=="E" and cell.Crystal()==4) else 1000
    p2 = -0.2 if (cell.iTED()=="E" and cell.Crystal()==4) else -1

    gaussFit = ROOT.TF1("gaussFit", "gaus(0)+pol1(3)", ADC_Low, ADC_High)
    gaussFit.SetParameters(TH1D.GetMaximum(),MaxBin,7,p1,p2,0)
    TH1D.Fit(gaussFit,"QR")
    
    sigma = abs(gaussFit.GetParameter(2))
    centroid_ch = gaussFit.GetParameter(1)
    
    x1 = cell(centroid_ch+sigma*numpy.sqrt(2*numpy.log(2)))
    x2 = cell(centroid_ch-sigma*numpy.sqrt(2*numpy.log(2)))
    centroid = cell(centroid_ch)
        
    fwhm = x1-x2
            
    return sigma*numpy.sqrt(2*numpy.log(2))*2/centroid_ch*100, (fwhm/centroid)*100, centroid

In [7]:
def TH1D_draw(cell):
    
    TH1D = cell.TH1D()
    
    canvas = ROOT.TCanvas()
    canvas.cd()
    
    TH1D.SetTitle(repr(cell))
    TH1D.SetStats(False)
    
    latex = ROOT.TLatex()
    latex.SetNDC()
    latex.SetTextSize(0.03)
    
    TH1D.Draw("pe")
    
    l1,l2,l3 = get_resolution(cell)

    ited = cell.iTED()
    
    #l4 = uproot.open(f"../../data/nTOF_March2022/888/CW100ns/Resolutions_Cs137_CenterScatter_iTED{ited}_8.8.8_100ns.root:grResolEnergy;1").values()[1][cell.Crystal()]
    
    latex.DrawText(0.7, 0.8, "R_ch: {:.2f}%".format(l1))
    
    latex.DrawText(0.7, 0.75, "R_E: {:.2f}%".format(l2))
        
    if 662*(1-l2/100) < l3 < 662*(1+l2/100):
        latex.DrawText(0.7, 0.7, "E: {:.0f}keV".format(l3))
    else:
        latex.DrawText(0.7, 0.7, "->E: {:.0f}keV".format(l3))
            
    return canvas

In [8]:
entries = []

for iTED in [4]:
    for Crystal in tqdm.tqdm([0,1,2,3,4]):
        for En in [511, 1274]:
                    
            cryst_code = ["A", "B", "C", "D", "E"][iTED]+str(Crystal)
                                                                    
            spectr = spectrum(
                file,
                ["A","B","C","D","E"][iTED], 
                Crystal,
                En, 
                calibrations_df.query("crystal == @cryst_code & configuration == '88C_CMAM'").iloc[0][[1,2,3]]
            )
                        
            entries.append(
                pandas.DataFrame({
                    "resolution": get_resolution(spectr)[1],
                    "fit": abs(get_resolution(spectr)[2]-En)/En*100,
                    "energy_measure": get_resolution(spectr)[2],
                    "energy_dif": get_resolution(spectr)[2]-En,
                    "cps": spectr.Rate(30),
                    "alpha": spectr.Alpha(30),
                    "iTED": ["A", "B", "C", "D", "E"][iTED],
                    "crystal": Crystal,
                    "energy": En,
                    "obj": spectr,
                }, index=[0])
            )
                    
entries_df = pandas.concat(entries, ignore_index=True)

100%|██████████████████| 5/5 [00:00<00:00, 13.74it/s]
Info in <TCanvas::MakeDefCanvas>:  created default TCanvas with name c1


## Analysis

### Resolution

#### per crystal per energy

In [9]:
entries_df.groupby(["energy","crystal"]).resolution.describe().drop(['count', 'std'], axis=1).T.style.background_gradient(cmap ='YlOrRd',axis=1)

### Fit

#### per crystal per energy

In [10]:
entries_df.groupby(["energy","crystal"]).fit.describe().drop(['count', 'std'], axis=1).T.style.background_gradient(cmap ='YlOrRd',axis=None)

In [11]:
entries_df.groupby(["energy","crystal"]).energy_measure.describe().drop(['count', 'std'], axis=1).T.style.background_gradient(cmap ='YlOrRd',axis=None)

In [12]:
entries_df.groupby(["energy","crystal"]).energy_dif.describe().drop(['count', 'std'], axis=1).T.style.background_gradient(cmap ='YlOrRd',axis=None)

### Count rate using integral

In [13]:
entries_df.groupby(["iTED","crystal"]).cps.describe().drop(['count', 'std'], axis=1).T.style.background_gradient(cmap ='YlOrRd',axis=1)

### Alpha activity rate using 1600keV < E < 2800keV

In [14]:
entries_df.groupby(["iTED","crystal"]).alpha.describe().drop(['count', 'std'], axis=1).T.style.background_gradient(cmap ='YlOrRd',axis=1)

## Draw fits

In [15]:
for index, row in entries_df.iterrows():
    TH1D_draw(row.obj).Draw()

In [16]:
calibrations_df

,Unnamed: 0,0,1,2,crystal,configuration,temperature_mean,temperature_min,temperature_max,temperature_change
0,6,115.4820,2.343750,0.000473,E0,88C_CMAM,24.61,24.6,24.8,0.2
1,7,87.1183,1.312900,0.001402,E0,88C_IFIC,24.34,24.3,24.4,0.1
2,0,181.9450,2.287200,0.000473,E1,88C_CMAM,24.83,24.8,25.0,0.2
3,1,252.1430,0.932148,0.001726,E1,88C_IFIC,24.60,24.6,24.6,0.0
4,8,161.2300,2.710730,0.000492,E2,88C_CMAM,24.99,24.8,25.0,0.2
5,9,121.9170,1.649660,0.001643,E2,88C_IFIC,24.72,24.6,24.8,0.2
6,4,221.7720,2.315470,0.000668,E3,88C_CMAM,24.39,24.3,24.4,0.1
7,5,279.8830,1.150800,0.001748,E3,88C_IFIC,24.09,23.9,24.1,0.2
8,2,180.8790,3.151310,0.000756,E4,88C_CMAM,25.70,25.7,25.7,0.0
9,3,155.3700,1.897040,0.002089,E4,88C_IFIC,25.28,25.2,25.3,0.1
